In [ ]:
from fastai.text.all import *

# Data

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
path.ls()

(#2) [Path('/home/renato/.fastai/data/imdb_sample/texts.csv'),Path('/home/renato/.fastai/data/imdb_sample/models')]

In [ ]:
df = pd.read_csv(path/'texts.csv')
df.sample(3)

,label,text,is_valid
594,positive,"The only explanation I can muster as to why this film isn't widely distributed is because it hits too close to home for some. This movie was a genuine happy surprise, the satire is genius. This film turns the lights on in the dark that is organized religion and big media, and the roaches scurry for cover. Rent the DVD and watch it for yourself if you haven't yet, this film succeeds where many have failed (Dogma comes to mind) to poke it's nose under the tent, both by using humor and very clever analogies coupled with telling backdrops and locations. Can't comment in depth without revealing...",False
958,negative,"If it were not for the ""Oh So Gourgous,"" Natassia Malthe, this B- movie would not have been worth one sector of my Tivo disk space! In what low rent, back lot warehouse was the supposed space port filmed in? ""Continuity People!"" It's a basic principle in real movie making! By night an alleged space port and by day (night and day on a space station?) a warehouse!??!? People Please! The only thing I will commend this movie for, is the wardrobe dept. for continuously, keeping Natassia in those tight shape revealing outfits! Even the women who saw this bomb had to appreciate the outfits that s...",True
522,positive,"Not sure if it was right or wrong, but I read thru the other comments before watching the short.I have to say I disagree with most of the negative comments or problems people have had with it.<br /><br />As a first time ""Lone Wolf"" director/producer,I like to see things that I can aspire to,not necessarily from the pro's, but by people just getting their feet wet like me.<br /><br />If indeed this is also from a first-timer,as I read,I applaud the effort.Marvelous job then in that respect! There were some comments about the music.I thought it was quite nice for the piece.Some say it kind o...",False


In [ ]:
dls = TextDataLoaders.from_df(df, text_col='text', label_col='label', valid_col='is_valid')

In [ ]:
dls.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n\n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , xxunk bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj victor xxmaj vargas became i was always aware that something did n't quite feel right . xxmaj victor xxmaj vargas suffers from a certain xxunk on the director 's part . xxmaj apparently , the director thought that the ethnic backdrop of a xxmaj latino family on the lower east side , and an xxunk storyline would make the film critic proof . xxmaj he was right , but it did n't fool me . xxmaj raising xxmaj victor xxmaj vargas is",negative
1,"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the xxunk and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with the xxunk possible scenarios to get the two protagonists together in the end . xxmaj in fact , all its charm is xxunk , contained within the characters and the setting and the plot … which is highly believable to xxunk . xxmaj it 's easy to think that such a love story , as beautiful as any other ever told , * could * happen to you … a feeling you do n't often get from other romantic comedies",positive
2,"xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj xxunk , after xxunk ) , i can xxunk join both xxunk of "" at xxmaj the xxmaj movies "" in taking xxmaj steven xxmaj soderbergh to task . \n\n xxmaj it 's usually satisfying to watch a film director change his style / subject , but xxmaj soderbergh 's most recent stinker , xxmaj the xxmaj girlfriend xxmaj xxunk ) , was also missing a story , so narrative ( and editing ? ) seem to suddenly be xxmaj soderbergh 's main challenge . xxmaj strange , after 20 - odd years in the business . xxmaj he was probably never much good at narrative , just xxunk it well inside "" edgy "" projects . \n\n xxmaj none of this excuses him this present ,",negative


## Model

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, metrics=accuracy, path=path).to_fp16()

In [ ]:
learn.model

SequentialRNN(
  (0): SentenceEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(7080, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(7080, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1152, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1152, 1152, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1152, 400, batch_first=True)
        )
      )
      (input_dp): RNNDropout()
      (hidden_dps): ModuleList(
        (0-2): 3 x RNNDropout()
      )
    )
  )
  (1): PoolingLinearClassifier(
    (layers): Sequential(
      (0): LinBnDrop(
        (0): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Dropout(p=0.2, inplace=False)
        (2): Linear(in_features=1200, out_features=50, bias=False)
        (3): ReLU(inplace=True)
      )
      (1): LinBnDrop(
        (0): Bat

In [ ]:
learn.load_encoder('lm_encoder')

<fastai.text.learner.TextLearner>

## Train

In [ ]:
learn.fit_one_cycle(5, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.527399,0.591638,0.660000,00:02
1,0.434701,0.464140,0.790000,00:02
2,0.380002,0.561624,0.780000,00:02
3,0.327309,0.496131,0.800000,00:02
4,0.283855,0.534622,0.780000,00:02


In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.170420,0.548937,0.805000,00:03
1,0.159448,0.596773,0.795000,00:03
2,0.138656,0.614346,0.825000,00:03
3,0.111848,0.553488,0.825000,00:03
4,0.091636,0.556082,0.830000,00:03
5,0.072780,0.533408,0.850000,00:03
6,0.058054,0.554837,0.850000,00:03
7,0.047723,0.553999,0.835000,00:03
8,0.039727,0.547975,0.845000,00:03
9,0.034183,0.543512,0.845000,00:03


In [ ]:
learn.show_results(max_n=3)

,text,category,category_
0,"xxbos xxmaj i 'm sure things did n't exactly go the same way in the real life of xxmaj homer xxmaj hickam as they did in the film adaptation of his book , xxmaj rocket xxmaj boys , but the movie "" october xxmaj sky "" ( an xxunk of the book 's title ) is good enough to stand alone . i have not read xxmaj hickam 's memoirs , but i am still able to enjoy and understand their film adaptation . xxmaj the film , directed by xxmaj joe xxmaj xxunk and written by xxmaj lewis xxmaj xxunk , xxunk the story of teenager xxmaj homer xxmaj hickam ( jake xxmaj xxunk ) , beginning in xxmaj october of 1957 . xxmaj it opens with the sound of a radio broadcast , bringing news of the xxmaj russian satellite xxmaj xxunk , the first artificial satellite in",positive,positive
1,"xxbos xxmaj to review this movie , i without any doubt would have to quote that memorable scene in xxmaj tarantino 's "" pulp xxmaj fiction "" ( xxunk ) when xxmaj jules and xxmaj vincent are talking about xxmaj mia xxmaj wallace and what she does for a living . xxmaj jules tells xxmaj vincent that the "" only thing she did worthwhile was pilot "" . xxmaj vincent asks "" what the hell is a pilot ? "" and xxmaj jules goes into a very well description of what a xxup tv pilot is : "" well , the way they make shows is , they make one show . xxmaj that show 's called a ' pilot ' . xxmaj then they show that show to the people who make shows , and on the strength of that one show they decide if they 're going to",negative,negative
2,"xxbos xxmaj how viewers react to this new "" adaption "" of xxmaj shirley xxmaj jackson 's book , which was promoted as xxup not being a remake of the original 1963 movie ( true enough ) , will be based , i suspect , on the following : those who were big fans of either the book or original movie are not going to think much of this one … and those who have never been exposed to either , and who are big fans of xxmaj hollywood 's current trend towards "" special effects "" being the first and last word in how "" good "" a film is , are going to love it . \n\n xxmaj things i did not like about this adaption : \n\n 1 . xxmaj it was xxup not a true adaption of the book . xxmaj from the xxunk i had",negative,negative
